In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/data/"))

# Any results you write to the current directory are saved as output.

['hockey', 'kabaddi', 'weight_lifting', 'table_tennis', 'cricket', 'ice_hockey', 'formula1', 'shooting', 'tennis', 'swimming', 'models', 'chess', 'football', 'wwe', 'motogp', 'boxing', 'gymnastics', 'volleyball', 'wrestling', 'badminton', 'fencing', 'baseball', 'basketball']


In [ ]:
!pip install imutils

In [41]:
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

In [42]:
# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = [
 'cricket',
 'formula1',
 'tennis']
LABELS

['cricket', 'formula1', 'tennis']

In [43]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images("../input/data/"))

[INFO] loading images...


In [44]:
data = []
labels = []

In [45]:
# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# if the label of the current image is not part of of the labels
	# are interested in, then ignore the image
	if label not in LABELS:
		continue

	# load the image, convert it to RGB channel ordering, and resize
	# it to be a fixed 224x224 pixels, ignoring aspect ratio
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

In [46]:
len(labels)

2055

In [47]:
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [50]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, stratify=labels, random_state=42)

# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [51]:
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [52]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [53]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / 50)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [54]:
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=32),
	steps_per_epoch=len(trainX) // 32,
	validation_data=valAug.flow(testX, testY),
	validation_steps=len(testX) // 32,
	epochs=50)

[INFO] training head...
Epoch 1/50
48/48 [==============================] - 24s 502ms/step - loss: 1.2062 - acc: 0.4232 - val_loss: 0.8681 - val_acc: 0.5879
Epoch 2/50
48/48 [==============================] - 17s 360ms/step - loss: 0.9269 - acc: 0.5702 - val_loss: 0.6219 - val_acc: 0.7469
Epoch 3/50
48/48 [==============================] - 18s 383ms/step - loss: 0.7525 - acc: 0.6809 - val_loss: 0.4843 - val_acc: 0.8320
Epoch 4/50
48/48 [==============================] - 18s 383ms/step - loss: 0.6466 - acc: 0.7321 - val_loss: 0.4251 - val_acc: 0.8838
Epoch 5/50
48/48 [==============================] - 18s 380ms/step - loss: 0.5498 - acc: 0.7836 - val_loss: 0.3729 - val_acc: 0.8963
Epoch 6/50
48/48 [==============================] - 18s 378ms/step - loss: 0.5154 - acc: 0.7999 - val_loss: 0.3421 - val_acc: 0.9066
Epoch 7/50
48/48 [==============================] - 18s 383ms/step - loss: 0.4699 - acc: 0.8300 - val_loss: 0.2967 - val_acc: 0.9253
Epoch 8/50
48/48 [===========================

In [55]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

     cricket       0.90      0.96      0.93       166
    formula1       0.96      0.96      0.96       169
      tennis       0.97      0.91      0.94       179

    accuracy                           0.94       514
   macro avg       0.94      0.94      0.94       514
weighted avg       0.94      0.94      0.94       514



In [58]:
# plot the training loss and accuracy
N = 50
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

In [60]:
plt.savefig('plot.png')

<a href='plot.png'>image</a>

In [62]:
# serialize the model to disk
print("[INFO] serializing network...")
model.save('activity.model')

[INFO] serializing network...


<a href='activity.model'>image</a>

In [63]:
# serialize the label binarizer to disk model/lb.pickle
f = open('lb.pickle', "wb")
f.write(pickle.dumps(lb))
f.close()

<a href='lb.pickle'>pickle</a>